In [22]:
from os.path import join, basename
import os
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
basepath = '/Users/gennosuke/Downloads'
coco_img_basename_path = [basename(i) for i in glob(join(basepath, 'synth', '200000_homo', '*'))]

In [20]:
def concat_tile(im_list_2d):
    """
    https://note.nkmk.me/python-opencv-hconcat-vconcat-np-tile/
    同じサイズの画像を縦・横にタイル状に連結
    """
    return cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in im_list_2d])

all = ['synth', 'arg', 'noarg']
homo = ['200000_homo', '200000_no_homo']
SAVE_DIR_PATH = 'result'
os.makedirs(SAVE_DIR_PATH, exist_ok=True)

for imgname in coco_img_basename_path:
    img_list = [[0 for _ in range(3)] for _ in range(2)]
    for i, key in enumerate(all):
        for j, h in enumerate(homo):
            img_path = join(basepath, key, h, imgname)
            img = cv2.imread(img_path)
            img = cv2.resize(img, dsize=(0, 0), fx=0.5, fy=0.5)
            img_list[j][i] = img
    img_tile = concat_tile(img_list)
    cv2.imwrite(f'{SAVE_DIR_PATH}/compare_{imgname}', img_tile)


In [39]:
def detect_feature_points_with_resize(img):
    img = cv2.resize(img, (960, 720))
    # AKAZE検出器を読み込む
    akaze = cv2.AKAZE_create()
    # 特徴点の検出
    kp = akaze.detect(img)
    for k in kp:
        cv2.circle(img, (int(k.pt[0]),int(k.pt[1])), radius=3, color=(0,255,0), thickness=-1)
    return img

In [43]:
def concat_tile(im_list_2d):
    """
    https://note.nkmk.me/python-opencv-hconcat-vconcat-np-tile/
    同じサイズの画像を縦・横にタイル状に連結
    """
    return cv2.hconcat(im_list_2d)

SAVE_DIR_PATH = 'result_with_akaze'
os.makedirs(SAVE_DIR_PATH, exist_ok=True)

for imgname in coco_img_basename_path:
    coco_img_path = join(basepath, 'val2014', imgname.replace('.png', '.jpg'))
    img = cv2.imread(coco_img_path)
    img_akaze = detect_feature_points_with_resize(img)
    magicpoint_homo_img_path = join(basepath, 'noarg', '200000_homo',imgname)
    magicpoint_homo_img = cv2.imread(magicpoint_homo_img_path)
    magicpoint_no_homo_img_path = join(basepath, 'noarg', '200000_no_homo',imgname)
    magicpoint_no_homo_img = cv2.imread(magicpoint_no_homo_img_path)
    img_list = [
        img_akaze,
        magicpoint_homo_img,
        magicpoint_no_homo_img
    ]
    img_tile = concat_tile(img_list)
    cv2.imwrite(f'{SAVE_DIR_PATH}/compare_with_akaze_{imgname}', img_tile)